In [ ]:
#coding= UTF-8

import sys, os, datetime, time, re
import random
from slackbot import bot

TOKEN_MYBOT = ######
bot.settings.API_TOKEN  = TOKEN_MYBOT
# testonetwothree 的 Channel ID 
CHANNEL_ID = 'C20QYLSKE' #"C1QQG5SJG"  
# 算分 Bot 的 ID
PIX_INSPECTOR = "U1QDCHJ3H"

from slackbot.bot import Bot
from slackbot.bot import respond_to
from slackbot.bot import listen_to

In [ ]:
def main():
    bot = Bot()
    bot.run()

In [ ]:
import gensim
model = gensim.models.Word2Vec.load('word2vec_combine_trained')

In [ ]:
import jieba

jieba.set_dictionary('traditional.dict.txt')
jieba.load_userdict('additional.dict.txt')

In [ ]:
f = open('Pix_sentence', 'r')
data_list = f.readlines()

In [ ]:
def bin_search(search_term, data_list):
    if len(search_term) <= 12:
        return 'DNE'
    
    min_v = 0
    max_v = len(data_list)
    mid = 0
    while(mid < max_v):
        mid = (min_v + max_v)/2
        
        if search_term in data_list[mid]: 
            return data_list[mid]

        elif search_term > data_list[mid]:
            min_v = mid+1
    
        else:
            max_v = mid
        mid = (min_v + max_v)/2
    
    return 'DNE'

In [ ]:
stopWords_set = set()
content = open('ch_stopwords.txt', 'rb').read().decode('utf-8')
for line in content.splitlines():
    stopWords_set.add(line.strip().encode('utf8'))
    stopWords_set.add(' ')

def replace_all(text, dic):
    for i, j in dic.iteritems():
        text = text.replace(i, j)
    return text
    
#this function returns the paragraph as a list of words(without the buzzword)
def get_cloze_par(par): 

    #par is the paragraph without any punctuation.
    reps = {'。':'', '!':'', ',':'', '.':'', '，':'', '～': '', '~':'', '︽⊙＿⊙︽':'BUZZWORD'}
    par = replace_all(par, reps)
    
    words = jieba.cut(par, cut_all=False)
    ans = list()
    for w in words:
        w = w.encode('UTF-8')
        if w not in stopWords_set and w in model:
            ans.append(w)
    return ans

In [ ]:
#get the highest score from the paragraph +1/10 of the entire paragraph
def get_answer(nums):
    ans = 0
    temp = nums[0]
    for i in range(len(nums)):
        if(nums[i] > temp):
            ans = i
            temp = nums[i]
    return ans

In [ ]:
def ans_score(nums):
    temp = nums[0]
    for i in range(len(nums)):
        if(nums[i] > temp):
            temp = nums[i]
    return temp

In [ ]:
def search_term(par):
    reps = {'。':'\n', '!':'\n', ',':'\n', '.':'\n', '，':'\n', '～': '/n', '~':'\n', '︽⊙＿⊙︽':'BUZZWORD'}
    par = replace_all(par, reps)
    sentences = par.splitlines()
    term = ''
    for i in range(len(sentences)):
        if 'BUZZWORD' in sentences[i]:
            sentence = sentences[i].strip()
    words = jieba.cut(sentence, cut_all=False)
    for w in words:
        w = w.strip()
        w = w.encode('UTF-8')
        if w == 'BUZZWORD':
            if len(term) == 0:
                return "DNE"
            return term
        else:
            term = term + w + ' '
    
    return term

In [ ]:
def relation_score2(a, b):
    s = a[0]
    g = 0
    for i in range(len(a)):
        if a[i] > s:
            s = a[i]
        g += a[i]
    s += g/10. + b
    return s

In [ ]:
def print_list(ans):
    w = ''
    for i in ans:
        w += i + ' '
    print w

In [ ]:
ANSWER_LIST = {}

## 取得 quizmaster 丟出的題目字串，解析出問題及選項
@listen_to(unicode("題目", 'utf-8') + ' (.*)', re.DOTALL)
def receive_question(message, question):
    
    pattern = re.compile('.*\[(\d+)\](.*)###(.*)\[END\]', re.DOTALL)  
    if message._client.users[message._get_user_id()][u'name'] == "pix_quizmaster" and pattern.match(question):
        m = re.match('.*\[(\d+)\](.*)###(.*)\[END\]', question, re.DOTALL)
        quiz_no = int(m.group(1).strip())
        question = m.group(2)
        options = {}
        for item in m.group(3).split(','):
            index, value = item.split(':')
            options[index.strip()] = value.strip()
        
        # 在此呼叫定義算法 
        choices = ['a', 'b', 'c', 'd', 'e']
        for index in choices:
            c = options[index]
        
        pot = []
        final_ans = ''
        
        search_ = search_term(question).strip() #find the cloze sentence
        search_w = search_.split() #make it into a list
        
        ans_ = bin_search(search_, data_list) #return cloze sentence in database
        if ans_ == 'DNE':
            ans = 'DNE'
        else:
            ans = ans_.split()[len(search_w)]
            
        for index in choices:
            c = options[index]
            if c.lower() == ans.lower():
                final_ans = index
                break
                
        if final_ans != '':
            ANSWER_LIST[quiz_no] = final_ans
        
        else:
            words = get_cloze_par(question)
            for index in choices:
                c = options[index]
                #假如選項不在 model 裡 用結巴斷句
                if c not in model: 
                    temp = jieba.lcut(c, cut_all=False)
                    c = temp[0].encode('UTF-8')
                    #假如斷完之後還是不在 model 中, 取第一個字
                    if c not in model:
                        c = c[0:3]
                    #英文字的話直接代換
                    if c not in model:
                        c = '嗨'
                        
                if ans == 'DNE':
                    add_ = 0
                else:
                    add_ = model.similarity(ans, c)
                b = model.similarity(words, c)
        
                pot.append(relation_score2(b, add_))
                ANSWER_LIST[quiz_no] = choices[get_answer(pot)]          

    #當 quizmaster 丟出 "機器人小朋友請搶答"，請儘速把答案丟到 channel     
@listen_to(unicode("機器人小朋友請搶答", 'utf-8') +'$')
def hello_send(message):
    if message._client.users[message._get_user_id()][u'name'] == "pix_quizmaster":
    
        reply_ans = ""
        for idx in ANSWER_LIST:
            reply_ans += str(idx) + " : " + ANSWER_LIST[idx] + ","
        message.send("<@%s>: %s %s" % (PIX_INSPECTOR, unicode("請給分 ", 'utf-8'), reply_ans[:-1]))
        ANSWER_LIST.clear()
        
# 幫按讚
@listen_to(unicode("題號", 'utf-8') + ' (.*)')
def hey(message, ans_string):
    message.react('+1')

# 回覆訊息範例
@respond_to('hello$', re.IGNORECASE)
def hello_reply(message):
    message.reply('hello sender!')


In [ ]:
main()